# Getting Started with Sea Ice CCI Data

### 1. Import Necessary Packages

In this section, we import the required Python packages to work with ESA Climate Change Initiative (CCI) data. Most notably, we use the [ESA Climate Toolbox](https://esa-climate-toolbox.readthedocs.io/en/latest/) which simplifies access, manipulation, and visualization of CCI datasets in Python.

These packages allow us to:
- Access satellite-based climate data records from ESA.
- Handle geospatial and temporal dimensions efficiently.
- Visualize data with intuitive plotting tools.

📚 For a broader introduction to the toolbox and how to install it, visit:  
🔗 [ESA CCI Climate Toolbox Quick Start](https://esa-climate-toolbox.readthedocs.io/en/latest/quick_start.html#jupyter-notebooks)  
🔗 [ESA Climate Data Toolbox Website](https://climate.esa.int/en/data/toolbox/)

In [1]:
from xcube.core.store import new_data_store
from esa_climate_toolbox.core import get_op
from esa_climate_toolbox.core import list_ecv_datasets
from esa_climate_toolbox.core import get_store
from esa_climate_toolbox.core import list_datasets
from esa_climate_toolbox.ops import plot
import xarray as xr
import matplotlib.pyplot as plt
import warnings
%matplotlib inline

# Suppress FutureWarnings related to xarray subclassing
warnings.filterwarnings("ignore", category=FutureWarning, message=".*xarray subclass VectorDataCube.*")
warnings.filterwarnings("ignore", category=FutureWarning)

<frozen abc>:106: FutureWarning: xarray subclass VectorDataCube should explicitly define __slots__


### Step 2: Connect to the ESA CCI Data Store

The ESA Climate Toolbox provides direct access to the **ESA Climate Data Store**, which hosts harmonized satellite-based climate data records produced under the **ESA Climate Change Initiative (CCI)**.

In this step, we establish a connection to the data store so we can browse and open datasets. This connection allows us to access data **without having to download files manually** — a convenient way to explore and analyze large geospatial datasets in cloud-friendly formats such as **Zarr** or **Kerchunk**.

The data store includes a wide range of essential climate variables (ECVs), such as aerosols, land surface temperature, sea level, and soil moisture.

📘 Learn more about available datasets:  
🔗 [ESA Climate Data Toolbox – Quick Start Guide](https://esa-climate-toolbox.readthedocs.io/en/latest/quick_start.html#data-access)

In [2]:
cci_store = new_data_store("esa-cci")
# List all available data sets of an ECV
# list_ecv_datasets("seaice")

[('esacci.SEAICE.day.L3C.SICONC.ESMR-(Nimbus-5).Nimbus-5.NIMBUS5_ESMR-EASE2_NH.1-0.NH',
  'esa-cci'),
 ('esacci.SEAICE.day.L3C.SICONC.ESMR-(Nimbus-5).Nimbus-5.NIMBUS5_ESMR-EASE2_SH.1-0.SH',
  'esa-cci'),
 ('esacci.SEAICE.day.L4.SICONC.multi-sensor.multi-platform.AMSR_25kmEASE2.2-1.NH',
  'esa-cci'),
 ('esacci.SEAICE.day.L4.SICONC.multi-sensor.multi-platform.AMSR_25kmEASE2.2-1.SH',
  'esa-cci'),
 ('esacci.SEAICE.day.L4.SICONC.multi-sensor.multi-platform.AMSR_50kmEASE2.2-1.NH',
  'esa-cci'),
 ('esacci.SEAICE.day.L4.SICONC.multi-sensor.multi-platform.AMSR_50kmEASE2.2-1.SH',
  'esa-cci'),
 ('esacci.SEAICE.day.L4.SICONC.multi-sensor.multi-platform.RE_SSMI_12-5kmEASE2-NH.3-0.NH',
  'esa-cci'),
 ('esacci.SEAICE.day.L4.SICONC.multi-sensor.multi-platform.RE_SSMI_12-5kmEASE2-SH.3-0.SH',
  'esa-cci'),
 ('esacci.SEAICE.mon.L3C.SITHICK.RA-2.Envisat.NH25KMEASE2.2-0.r1', 'esa-cci'),
 ('esacci.SEAICE.mon.L3C.SITHICK.RA-2.Envisat.NH25KMEASE2.3-0.r1', 'esa-cci'),
 ('esacci.SEAICE.mon.L3C.SITHICK.RA-2.En

### Step 3: Define the Dataset ID  
To work with a specific ESA CCI dataset, we need to specify its **dataset ID**. This unique identifier tells the toolbox which variable and product we want to access.

In this example, we are using a dataset from the **Sea Ice CCI project** that provides daily Level 4 sea ice concentration (SICONC) data derived from multiple satellite sensors over the Southern Hemisphere. Sea ice concentration is a key parameter for monitoring polar climate change, seasonal variability, and its interaction with the atmosphere and ocean.

For other ESA CCI projects (e.g., Ozone, Land Cover, Permafrost), simply replace the dataset ID accordingly.

We will use the following dataset ID:

In [3]:
data_id = 'esacci.SEAICE.day.L4.SICONC.multi-sensor.multi-platform.AMSR_25kmEASE2.2-1.SH'

📘 A full list of dataset IDs can be retrieved from the store or found in the [ESA CCI Climate Toolbox documentation](https://esa-climate-toolbox.readthedocs.io/en/latest/data_access.html#available-datasets).

### Step 4: Describe Dataset (Check Available Variables and Metadata)

Before loading the full dataset, it’s helpful to **inspect the metadata** to understand its structure. This includes:

- Available variables (e.g., AOD, uncertainty estimates)
- Temporal and spatial coverage
- Data format and structure

This step ensures we know what the dataset contains and how to work with it. It also helps confirm that the variable we want to plot or analyze is actually included.

🛠️ Tip: You can use the description to verify variable names, dimensions (e.g., `lat`, `lon`, `time`), and time coverage.

📘 More on dataset structure:  
🔗 [ESA Climate Toolbox – Data Access](https://esa-climate-toolbox.readthedocs.io/en/latest/data_access.html)

In [4]:
cci_store.describe_data(data_id)

### Step 5: Check Open Parameters for the Dataset

Before opening the dataset, we can inspect which parameters are supported by the **Zarr opener** (e.g., `time_range`, `bbox`, `variable_names`). This step helps ensure that we pass valid arguments when loading data and avoid errors.

The command below lists all expected input parameters and their allowed values for the selected dataset.

In [5]:
cci_store.get_open_data_params_schema(data_id=data_id, opener_id='dataset:zarr:cciodp')

### Step 6: Define Region, Time Range, and Variables of Interest

Before opening the dataset, we define a few key parameters:

- **Time range**: the date(s) we want to load
- **Variables**: which data variable(s) to retrieve
- *(Optional)* **Bounding box**: spatial region of interest — here we skip it to load the global dataset

In [6]:
variables = ['ice_conc']  # Variable to retrieve
start_date = '2017-09-21'    # Start and end date (same for a single timestep)
end_date = '2017-09-21'
# bbox = (-10.0, 35.0, 30.0, 60.0)  # Optional: restrict to a region like Europe

### Step 7: Open the Dataset

Now we open the dataset using the selected parameters.  
The ESA Climate Toolbox will download only the necessary data (e.g., variable and time range).
You can always adjust the time range or variables to explore different slices of the dataset.

In [7]:
seaice_ds = cci_store.open_data(
    data_id=data_id,
    variable_names=variables,
    time_range=[start_date, end_date]
    # bbox=bbox  # Uncomment if regional selection is needed
)

### Step 8: Display Dataset Structure

We print a summary of the opened dataset to inspect its structure, dimensions, variables, and metadata.  
This helps verify that the data was loaded correctly and shows what is available for analysis and visualization.
This step is useful to understand what the dataset contains before working with it further.

In [8]:
print("\nOpened Dataset:\n", seaice_ds)


Opened Dataset:
 <xarray.Dataset> Size: 1MB
Dimensions:                 (time: 1, yc: 432, xc: 432, bnds: 2)
Coordinates:
  * time                    (time) datetime64[ns] 8B 2017-09-21T12:00:00
    time_bnds               (time, bnds) datetime64[ns] 16B dask.array<chunksize=(1, 2), meta=np.ndarray>
  * xc                      (xc) float64 3kB -5.388e+03 -5.362e+03 ... 5.388e+03
  * yc                      (yc) float64 3kB 5.388e+03 5.362e+03 ... -5.388e+03
Dimensions without coordinates: bnds
Data variables:
    Lambert_Azimuthal_Grid  float64 8B ...
    ice_conc                (time, yc, xc) float64 1MB dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    title:                   esacci.SEAICE.day.L4.SICONC.multi-sensor.multi-p...
    date_created:            2025-04-14T13:48:30.277255
    processing_level:        L4
    time_coverage_start:     2017-09-21T00:00:00
    time_coverage_end:       2017-09-22T00:00:00
    time_coverage_

### Step 9: Visualize Results

We now create a simple map plot of the selected variable.  
This allows us to explore the spatial patterns of the data — in this case, the Aerosol Optical Depth (AOD) for the selected day.
For more interactive and advanced visualizations, check out the [ESA Climate Toolbox](https://climate.esa.int/en/data/toolbox/) or the [Toolbox documentation](https://esa-climate-toolbox.readthedocs.io/en/latest/quick_start.html#jupyter-notebooks).

In [12]:
# Force eager loading by disabling chunks
seaice_ds_eager = seaice_ds.compute()

# Extract ice concentration
ice_conc = seaice_ds_eager["ice_conc"].isel(time=0).squeeze()

# Extract coordinates
x = seaice_ds_eager["xc"].values
y = seaice_ds_eager["yc"].values

# Plot
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.SouthPolarStereo())

mesh = ax.pcolormesh(
    x, y, ice_conc,
    transform=ccrs.SouthPolarStereo(),
    cmap="Blues",
    vmin=0, vmax=100
)

ax.coastlines()
plt.colorbar(mesh, ax=ax, orientation='vertical', label="Sea Ice Concentration (%)")
plt.title("Sea Ice Concentration – Southern Hemisphere")
plt.tight_layout()
plt.show()

KeyError: "ice_conc/0.0.0: cannot fetch chunk for variable 'ice_conc' and time_range (Timestamp('2017-09-21 00:00:00'), Timestamp('2017-09-22 00:00:00'))."

### Another try to plot it

In [13]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Drop time dimension early and convert to NumPy
ice_conc = seaice_ds["ice_conc"].isel(time=0).squeeze()

# Converting to a NumPy array directly, without .load()
ice_conc_np = ice_conc.data.compute()

# Coordinates
x = seaice_ds["xc"].values
y = seaice_ds["yc"].values

# Plot
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.SouthPolarStereo())

mesh = ax.pcolormesh(
    x, y, ice_conc_np,
    transform=ccrs.SouthPolarStereo(),
    cmap="Blues",
    vmin=0, vmax=100
)

ax.coastlines()
plt.colorbar(mesh, ax=ax, orientation='vertical', label="Sea Ice Concentration (%)")
plt.title("Sea Ice Concentration – Southern Hemisphere")
plt.tight_layout()
plt.show()

KeyError: "ice_conc/0.0.0: cannot fetch chunk for variable 'ice_conc' and time_range (Timestamp('2017-09-21 00:00:00'), Timestamp('2017-09-22 00:00:00'))."